In [1]:
import pandas as pd
import numpy as np 

In [3]:
# Hierarchical indexing
# We can create a multi-index from the tuples
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
print(pop)
index = pd.MultiIndex.from_tuples(index)
index

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64


MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           codes=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [5]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [8]:
# Multiindex as extra Dimension
pop_df = pop.unstack()
pop_df
# stack method provides the opposite of unstack()


,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [9]:
pop_df = pd.DataFrame({'total': pop,
                        'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [14]:
pop_df.unstack()

total            under18         
                2000      2010     2000     2010
California  33871648  37253956  9267089  9284094
New York    18976457  19378102  4687374  4318033
Texas       20851820  25145561  5906301  6879014

In [15]:
# Methods of multi-index creation
# implitic creation
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.041572  0.414332
  2  0.575370  0.329158
b 1  0.370215  0.749673
  2  0.018821  0.150128

In [17]:
# Explicit MulitIndex constructors
# 1) from_Arrays
pd.MultiIndex.from_arrays([['a','a','b','b'],[1,2,1,2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [18]:
# 2) from_tuples
pd.MultiIndex.from_tuples([('a',1),('a',2),('b',1),('b',2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [19]:
# 3) from_product
pd.MultiIndex.from_product([['a','b'],[1,2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [21]:
# MultiIndex level names
pop.index.names = ['state','year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [28]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      41.0  38.4  27.0  38.9  45.0  38.3
     2      40.0  36.8  49.0  37.1  45.0  37.8
2014 1      33.0  36.5  36.0  36.3  25.0  38.3
     2      25.0  37.6  24.0  36.7  32.0  38.4

In [29]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      27.0  38.9
     2      49.0  37.1
2014 1      36.0  36.3
     2      24.0  36.7

In [36]:
# Indexing and Slicing a MultiIndex
# 1) multi index Series
print(pop['California',2010])
print(pop.loc['California'], end="\n\n")
print(pop.loc['California':'New York'])
print(pop[pop>22000000])

37253956
year
2000    33871648
2010    37253956
dtype: int64

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64
state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64


In [37]:
# 2) Multiply indexed DataFrame
# remember that columns are primary in a dataframe and the syntax used for multiply indexed series applied to the columns
health_data['Guido','HR']
# health_data[:2,:2]

year  visit
2013  1        27.0
      2        49.0
2014  1        36.0
      2        24.0
Name: (Guido, HR), dtype: float64

In [38]:
# in loc or iloc index can be passed as tuple of multiple indices
health_data.loc[:, ("Bob", 'Temp')]

year  visit
2013  1        38.4
      2        36.8
2014  1        36.5
      2        37.6
Name: (Bob, Temp), dtype: float64

In [41]:
# Working with slices within these index tuples is not especially convenient; trying to create a slice within a tuple will lead to a syntax error:
health_data.loc[:,(:"HR")]

SyntaxError: invalid syntax (<ipython-input-41-dfb032bf0db3>, line 2)

In [45]:
# A better way is to use IndexSlice object which pandas provides for precisely for this situation
idx = pd.IndexSlice
health_data.loc[idx[:,1], idx[:, "HR"]]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,41.0,27.0,45.0
2014,1,33.0,36.0,25.0


In [47]:
# Rearranging Multi-Indices
# Sorted and Unsorted indices
index = pd.MultiIndex.from_product([['a','c','b'],[1,2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.247317
      2      0.141343
c     1      0.698626
      2      0.075649
b     1      0.861292
      2      0.541994
dtype: float64

In [49]:
# if we try to take a partial slice of the indices which are not sorted lexographically then it will generate a keyerror
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


In [52]:
data = data.sort_index()
data['a':'b']

char  int
a     1      0.247317
      2      0.141343
b     1      0.861292
      2      0.541994
dtype: float64

In [56]:
# Index setting and resetting
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [57]:
pop_flat.set_index(['state','year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

In [59]:
# Data Aggregation on Multi-indices
data_mean = health_data.mean(level="year")
data_mean

subject   Bob        Guido         Sue       
type       HR   Temp    HR  Temp    HR   Temp
year                                         
2013     40.5  37.60  38.0  38.0  45.0  38.05
2014     29.0  37.05  30.0  36.5  28.5  38.35

In [60]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,41.166667,37.883333
2014,29.166667,37.300000
